In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Data Exploration
- Let's extract the data first and concat the two json file into single csv file. Save the csv file
- Data exploration afterwards

In [2]:
# df = pd.read_json('gumtree.json')
# df2 = pd.read_json('remaininggumtree.json')
# data = pd.concat([df,df2])
# #Saving it as a final_data.csv
# data.to_csv('final_data.csv',index=False)

In [3]:
dataset = pd.read_csv('finals.csv')

In [4]:
#Let's check out the coloumn names
dataset.columns #All the coloumns here are important so we won't be deleting it. 

Index(['Unnamed: 0', 'Year', 'Body type', 'Mileage', 'Transmission',
       'Brochure Engine size', 'Fuel type', 'Urban mpg', 'Extra Urban mpg',
       'Fuel Consumption', 'Insurance Group', 'CO2 emissions',
       'Euro Emissions', 'Engine Power', 'Engine size',
       'Acceleration (0-62mph)', 'Top Speed', 'Seats', 'Doors', 'Colour',
       'Fuel Capacity', 'Luggage Capacity (seats up)', 'Price', 'Name',
       'Location'],
      dtype='object')

In [5]:
#However some of the coloumns needs some renaming to be done 
#The reason we are doing this is because we want
dataset.rename({'Acceleration (0-62mph)':'Acceleration(0-62mph)','Luggage Capacity (seats up)':'Luggage Capacity(seats_up)'},inplace=True)

In [6]:
#Checking if there is any duplicate values in the dataset
#Let's find out duplicate values
dataset.duplicated().sum() #There are alltogether 300 values that are duplicate
#Let's see what those duplicate values are
duplicates = dataset[dataset.duplicated()]
#duplicates.to_csv('duplicates.csv',index=False) #Exporting to csv to further analyse the data properly.(Not necessary)

In [7]:
#Let's remove the duplciates from the main dataset
final_dataset = dataset.drop_duplicates()

In [8]:
#Let's work with the dataset now that the duplicates has been removed. 
#We see alot of blank data so let's replace the blank data points with NaN
final_dataset = final_dataset.replace(r'^\s*$',np.nan,regex = True) #Using regex

#Let's also remove unwanted spaces from the string coloumn name
final_dataset['Name'] = final_dataset['Name'].str.strip()

In [9]:
#final_dataset.to_csv('finals.csv',index=False) #We sucessfully removed all the 300 duplicates from the data. 
#Before the data dimanesion was 1078 rows × 24 columns and now after removing 300 duplicate values, it is 778 rows × 24 columns

In [10]:
#Let's remove the unwanted columns
final_dataset.drop(['Urban mpg','Extra Urban mpg'],axis=1,inplace=True)

In [11]:
#Let's clean coloumns that needs cleaning
def remove_html_tags(string):
    result = re.sub('</h1','',string)
    return result

def remove_emojis(col): #Upon careful looking at the dataset carefully, there is an emoji '💥' that is in dataset s
    col = col.apply(lambda x: x.encode('ascii','ignore').decode('ascii')) #let's remove it. 
    return col

final_dataset['Name'] = final_dataset['Name'].apply(lambda x : remove_html_tags(x))
final_dataset['Name'] = remove_emojis(final_dataset['Name']) 

In [ ]:
#Let's seperate the dataset by creating a different variable for only the empty borchure size
empty_brochure = final_dataset[final_dataset['Brochure Engine size'].isna()] 

#Let's remove the nan file with the one we googled.
empty_brochure['Brochure Engine size'][empty_brochure.Name=='Toyota Hilux 3.0l Invincible'] = '3.0L' #For the Toyota
empty_brochure['Brochure Engine size'][empty_brochure.Name=='2011 Volkswagen Golf Match TDI 1.6 105 PS Hatchback Diesel Manual'] = '1.6L' #For volkswagen hatchback
empty_brochure['Brochure Engine size'][empty_brochure.Name=='2021 Citroen C4 50kWh Shine Plus Auto 5dr Hatchback Electric Automatic'] = 'Electric engine' #For Citroen C4
empty_brochure['Brochure Engine size'][empty_brochure.Name=='2021 Nissan Navara Brand new 2121 model  Double Cab Pick Up N-Guard in stock  wi'] = '2.3L' #For volkswagen hatchback

#Merge the final dataset with updated empty brochure updated data
final_dataset.update(empty_brochure) #It has been updated


In [13]:
fuel_consumption = final_dataset[final_dataset['Fuel Consumption'].isna()]
fuel_consumption

,Unnamed: 0,Year,Body type,Mileage,Transmission,Brochure Engine size,Fuel type,Fuel Consumption,Insurance Group,CO2 emissions,...,Acceleration (0-62mph),Top Speed,Seats,Doors,Colour,Fuel Capacity,Luggage Capacity (seats up),Price,Name,Location
129,208.0,2011.0,Hatchback,"105,496miles",Manual,1.6L,Diesel,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Blue,NaN,NaN,"£4,295.00",2011 Volkswagen Golf Match TDI 1.6 105 PS Hatc...,"Peterborough, Cambridgeshire"
295,445.0,2021.0,Hatchback,500miles,Automatic,Electric engine,Electric,NaN,22P,0g/km,...,9seconds,93mph,5.0,5.0,Blue,NaN,380litres,"£29,995.00",2021 Citroen C4 50kWh Shine Plus Auto 5dr Hatc...,"Chiswick, London"
328,488.0,2020.0,Saloon,"4,358miles",Automatic,0.0L,Electric,NaN,50D,0g/km,...,4.2seconds,145mph,5.0,4.0,White,NaN,542litres,"£47,500.00",2020 Tesla Model 3 Dual Motor Long Range Auto ...,"Chelmsford, Essex"
556,768.0,2014.0,Hatchback,"41,810miles",Automatic,0.0L,Other,NaN,24E,0g/km,...,11.5seconds,89mph,5.0,5.0,Grey,NaN,355litres,"£8,295.00",2014 64 NISSAN LEAF 0.0 TEKNA 5D 109 BHP 24 KW...,"Preston, Lancashire"
562,774.0,2015.0,Hatchback,"31,602miles",Automatic,0.0L,Electric,NaN,24E,0g/km,...,11.5seconds,89mph,5.0,5.0,Red,NaN,355litres,"£8,407.00",2015 Nissan Leaf 80kW Tekna 24kWh 5dr Auto HAT...,"St George, Bristol"
610,831.0,2020.0,Estate,"9,120miles",Automatic,2.0L,Diesel,NaN,17E,139g/km,...,9.4seconds,124mph,5.0,5.0,Grey,60litres,615litres,"£31,000.00",2020 Volkswagen Tiguan 2.0 TDi 150 4Motion R-L...,"Cannock, Staffordshire"
708,981.0,2020.0,Other,"3,891miles",Automatic,1.0L,Electric,NaN,26E,0g/km,...,9.6seconds,93mph,5.0,5.0,Blue,NaN,341litres,"£23,500.00",2020 Volkswagen Golf 99kW e-Golf 35kWh 5dr Aut...,"Castle Donington, Derbyshire"
770,1070.0,2020.0,Hatchback,"19,000miles",Manual,1.6L,Diesel,NaN,9E,97g/km,...,11.2seconds,115mph,5.0,5.0,Blue,40litres,351litres,"£13,885.00",2020 Volkswagen Polo 1.6 TDI SEL (s/s) 5dr Hat...,"Fareham, Hampshire"


In [14]:
#Let's work on fuel consumption empty data and turning it into dictionary
name = list(fuel_consumption['Name'])
consumption = ['62.8mpg','0mpg','0mpg','0mpg','0mpg','60mpg','0.0mpg','65.7mpg'] #Co
consumption_dict = dict(zip(name,consumption))

In [31]:
for i,j in consumption_dict.items():
    for k in fuel_consumption['Name']:
        if i==k:
            print(i)

2011 Volkswagen Golf Match TDI 1.6 105 PS Hatchback Diesel Manual
2021 Citroen C4 50kWh Shine Plus Auto 5dr Hatchback Electric Automatic
2020 Tesla Model 3 Dual Motor Long Range Auto 4WDE 4dr Saloon Electric Automatic
2014 64 NISSAN LEAF 0.0 TEKNA 5D 109 BHP 24 KWH BATTERY OWNED
2015 Nissan Leaf 80kW Tekna 24kWh 5dr Auto HATCHBACK Electric Automatic
2020 Volkswagen Tiguan 2.0 TDi 150 4Motion R-Line Tech 5dr DSG Auto Estate Diese
2020 Volkswagen Golf 99kW e-Golf 35kWh 5dr Auto Electric Automatic
2020 Volkswagen Polo 1.6 TDI SEL (s/s) 5dr Hatchback Diesel Manual


In [13]:
#Looks like there are alot of missing values that we need to handle. The coloumns with missing values are
#Brochure Engine Size
#Urban mpg
#Extra Urban mpg
#Insurance group
#C02 emissions
#Euro Emissions
#Engine Power
#Acceleration
#Top Speed
#Seats
#Doors
#Fuel Capacity
#Luggage Capacity
#Let's work on the missing values one by one. The good thing about missing values in car is that you can simply manually google it

In [14]:
#Brochure engine size
#Before googling the data, we need to check which rows of brochure engine size datas are missing from and which car it belongs to
# The four missing values for borchure are 
#2008 Toyota Hilux 3.0l Invincible ---> 3.0L
#2011 Volkswagen golf match TDI 1.6 105 PS HatchBack -->1.6L
#2021 Citroen C4 50kWh Shine Plus Auto 5dr HatchBack --Electric
#2021 Nissan Navara Brand new 2121 model Double hatchback-->2.3L

#Looks like some of the empty brochure size is of that of electric cars, so let's clean that aswell